In [6]:
from scalar_metrics_v02 import LoadMetrics
import numpy as np

In [7]:
cam, exp, night = 'r3', '00000003', '20190101'
lm = LoadMetrics(cam, exp, night)

countpix loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-countpix-r3-00000003.yaml
getbias loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-getbias-r3-00000003.yaml
getrms loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-getrms-r3-00000003.yaml
xwsigma loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-xwsigma-r3-00000003.yaml
countbins loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-countbins-r3-00000003.yaml
integ loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-integ-r3-00000003.yaml
skycont loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-skycont-r3-00000003.yaml
skypeak loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-skypeak-r3-00000003.yaml
skyresid loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-skyresid-r3-00000003.yaml

In [8]:
metrics, tests  = lm.metrics, lm.tests # Load_metrics_n_tests()

countpix  = metrics['countpix']
getbias   = metrics['getbias']
getrms    = metrics['getrms']
xwsigma   = metrics['xwsigma']
countbins = metrics['countbins']
integ     = metrics['integ']
skycont   = metrics['skycont']
skypeak   = metrics['skypeak']
skyresid  = metrics['skyresid']
snr       = metrics['snr']

In [41]:
def hist_bk(metric, key, loghist=False):
    from bokeh.models import HoverTool, ColumnDataSource
    bins = np.sqrt(len(metric[key]))/1.5
    bins = int(bins)
    hist, edges = np.histogram(metric[key], bins = bins)
    
    logcount = []
    for xi in hist:
        if xi >0: 
            logcount.append(np.log10(xi))
        else:
            logcount.append(xi)
            
    source = ColumnDataSource(data=dict(
        counts = list(hist),
        logcounts = logcount,
        left = edges[:-1],
        right = edges[1:],
        bottom = [0]*len(hist),
        ))

    hover = HoverTool(tooltips=[
         ("counts", "@counts"),
         ])
    
    p = figure(title=key,tools=[hover],
            background_fill_color="white")
    if loghist:
        p.quad(top="logcounts", source=source, bottom="bottom", left="left", right="right",
                fill_color="dodgerblue", line_color="black")
        p.yaxis.axis_label = 'Log10(COUNTS)'       
    else:
        p.quad(top="logcounts", source=source, bottom="bottom", left="left", right="right",
                fill_color="dodgerblue", line_color="black")
        p.yaxis.axis_label = 'COUNTS'


    p.legend.location = "top_left"
    p.xaxis.axis_label = key
    return p

In [42]:
p1 = hist_bk(countbins,"NBINSLOW", loghist=True);
p2 = hist_bk(countbins,"NBINSHIGH", loghist=True);
p3 = hist_bk(xwsigma,"XSIGMA");
p4 = hist_bk(xwsigma,"WSIGMA", loghist=True)


# xwsigma, "XSIGMA", "WSIGMA"
# countbins['NBINSHIGH'], bins=12,
# countbins['NBINSLOW'], bins=12, 
# countbins['NBINSMED']
#show(gridplot(p1,p2, ncols=2, plot_width=350, plot_height=350, toolbar_location="left"))


In [43]:
show(gridplot(p1,p2, p3, p4, ncols=1))#, plot_width= 350, plot_height=300))


In [38]:
np.sqrt(500)/15.

1.4907119849998598

# Outro

In [11]:
import numpy as np
import scipy.special

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_notebook

output_notebook()

p1 = figure(title="Normal Distribution (μ=0, σ=0.5)",tools="save",
            background_fill_color="#E8DDCB")

mu, sigma = 0, 0.5

measured = np.random.normal(mu, sigma, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(-2, 2, 1000)
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
p1.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend="PDF")
p1.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="CDF")

p1.legend.location = "top_left"
p1.xaxis.axis_label = 'x'
p1.yaxis.axis_label = 'Pr(x)'



p2 = figure(title="Log Normal Distribution (μ=0, σ=0.5)", tools="save",
            background_fill_color="#E8DDCB")

mu, sigma = 0, 0.5

measured = np.random.lognormal(mu, sigma, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(0, 8.0, 1000)
pdf = 1/(x* sigma * np.sqrt(2*np.pi)) * np.exp(-(np.log(x)-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((np.log(x)-mu)/(np.sqrt(2)*sigma)))/2

p2.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
p2.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend="PDF")
p2.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="CDF")

p2.legend.location = "bottom_right"
p2.xaxis.axis_label = 'x'
p2.yaxis.axis_label = 'Pr(x)'



p3 = figure(title="Gamma Distribution (k=1, θ=2)", tools="save",
            background_fill_color="#E8DDCB")

k, theta = 1.0, 2.0

measured = np.random.gamma(k, theta, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(0, 20.0, 1000)
pdf = x**(k-1) * np.exp(-x/theta) / (theta**k * scipy.special.gamma(k))
cdf = scipy.special.gammainc(k, x/theta) / scipy.special.gamma(k)

p3.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
p3.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend="PDF")
p3.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="CDF")

p3.legend.location = "top_left"
p3.xaxis.axis_label = 'x'
p3.yaxis.axis_label = 'Pr(x)'



p4 = figure(title="Beta Distribution (α=2, β=2)", tools="save",
            background_fill_color="#E8DDCB")

alpha, beta = 2.0, 2.0

measured = np.random.beta(alpha, beta, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(0, 1, 1000)
pdf = x**(alpha-1) * (1-x)**(beta-1) / scipy.special.beta(alpha, beta)
cdf = scipy.special.btdtr(alpha, beta, x)

p4.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
p4.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend="PDF")
p4.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="CDF")

p4.xaxis.axis_label = 'x'
p4.yaxis.axis_label = 'Pr(x)'



p5 = figure(title="Weibull Distribution (λ=1, k=1.25)", tools="save",
            background_fill_color="#E8DDCB")

lam, k = 1, 1.25

measured = lam*(-np.log(np.random.uniform(0, 1, 1000)))**(1/k)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(0, 8, 1000)
pdf = (k/lam)*(x/lam)**(k-1) * np.exp(-(x/lam)**k)
cdf = 1 - np.exp(-(x/lam)**k)

p5.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
       fill_color="#036564", line_color="#033649")
p5.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend="PDF")
p5.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="CDF")

p5.legend.location = "top_left"
p5.xaxis.axis_label = 'x'
p5.yaxis.axis_label = 'Pr(x)'



#output_file('histogram.html', title="histogram.py example")

show(gridplot(p1,p2,p3,p4,p5, ncols=2, plot_width=400, plot_height=400, toolbar_location=None))

Loading BokehJS ...

/home/foliveira/miniconda3/envs/quicklook/lib/python3.5/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in true_divide
/home/foliveira/miniconda3/envs/quicklook/lib/python3.5/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in log
/home/foliveira/miniconda3/envs/quicklook/lib/python3.5/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in multiply
/home/foliveira/miniconda3/envs/quicklook/lib/python3.5/site-packages/ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in log


In [42]:
    
    
hist, edges = np.histogram(xwsigma['XSIGMA'], bins = 17)

p = figure(title="XSIGMA",tools="hover",
            background_fill_color="white")

p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="dodgerblue", line_color="black")

p.legend.location = "top_left"
p.xaxis.axis_label = 'XSIGMA'
p.yaxis.axis_label = 'COUNTS'
show(p)

print( hist)

[ 1  5 13 13 37 36 60 90 77 60 47 25 16  7  8  4  1]


In [17]:


p1 = figure(title="Normal Distribution (μ=0, σ=0.5)",tools="save",
            background_fill_color="white")

mu, sigma = 0.7, 0.5

measured = np.random.normal(mu, sigma, 1000)
hist, edges = np.histogram(measured, density=True, bins=50)

x = np.linspace(-2, 2, 1000)
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="lightgreen", line_color="black")
#p1.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend="PDF")
#p1.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="CDF")

p1.legend.location = "top_left"
p1.xaxis.axis_label = 'x'
p1.yaxis.axis_label = 'Pr(x)'

show(p1)